In [1]:
from os import path
import pandas as pd
from serpapi import GoogleSearch
import datetime

In [2]:
search_term = "data analyst"
search_location = "Geneva, Switzerland"
search_radius = 20
api_key = open("../API keys/serpai.txt", "r").read()


for num in range(45):

    start = num * 10
    params = {
        "api_key": api_key,
        "device": "desktop",
        "engine": "google_jobs",
        "google_domain": "google.com",
        "q": search_term,
        "hl": "en",
        "gl": "ch",
        "lrad": search_radius,
        "location": search_location,
        #"chips": "date_posted:today",
        "start": start,
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    # check if the last search page (i.e., no results)
    try:
        if results['error'] == "Google hasn't returned any results for this query.":
            break
    except KeyError:
        print(f"Getting SerpAPI data for page: {start}")
    else:
        continue

    # create dataframe of 10 pulled results
    jobs = results['jobs_results']
    jobs = pd.DataFrame(jobs)
    jobs = pd.concat([pd.DataFrame(jobs), 
                    pd.json_normalize(jobs['detected_extensions'])], 
                    axis=1)
    jobs['date_time'] = datetime.datetime.utcnow()

    # concat dataframe
    if start == 0:
        jobs_all = jobs
    else:
        jobs_all = pd.concat([jobs_all, jobs])

    jobs_all['search_term'] = search_term
    jobs_all['search_location'] = search_location

Getting SerpAPI data for page: 0
Getting SerpAPI data for page: 10
Getting SerpAPI data for page: 20
Getting SerpAPI data for page: 30
Getting SerpAPI data for page: 40
Getting SerpAPI data for page: 50
Getting SerpAPI data for page: 60


In [3]:
jobs_all.drop(columns='detected_extensions',inplace=True)

In [4]:
jobs_all

,title,company_name,location,via,description,job_highlights,related_links,extensions,job_id,thumbnail,posted_at,schedule_type,date_time,search_term,search_location
0,Data Analyst,World Trade Organization WTO - Organisation Mo...,Geneva,via Npo-Jobs.ch,VN Category (INT / EXT / EOI):\nExternal\n\nAp...,[{'items': ['VN Category (INT / EXT / EOI): Ex...,"[{'link': 'http://www.wto.org/', 'text': 'wto....","[15 hours ago, Full-time and Temp work]",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,NaN,15 hours ago,Full-time and Temp work,2024-06-04 12:33:40.843725,data analyst,"Geneva, Switzerland"
1,Data Analyst,Organisation Météorologique Mondiale - OMM,Geneva,via Geneve-Jobs.ch,"ORGANIZATIONAL UNIT\n\nMonitoring, Evaluation,...","[{'items': ['ORGANIZATIONAL UNIT Monitoring, ...","[{'link': 'http://www.wmo.int/', 'text': 'wmo....","[5 days ago, Full-time]",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QiLCJodGlkb2...,NaN,5 days ago,Full-time,2024-06-04 12:33:40.843725,data analyst,"Geneva, Switzerland"
2,Client Data Analyst - Private Banking - 6 mont...,Michael Page,Geneva,via Michael Page Switzerland,About Our Client\n\nWe are seeking a detail-or...,[{'items': ['About Our Client We are seeking ...,"[{'link': 'http://www.page.com/', 'text': 'pag...","[23 days ago, Temp work]",eyJqb2JfdGl0bGUiOiJDbGllbnQgRGF0YSBBbmFseXN0IC...,https://encrypted-tbn0.gstatic.com/images?q=tb...,23 days ago,Temp work,2024-06-04 12:33:40.843725,data analyst,"Geneva, Switzerland"
3,Data Analyst R&D,Capgemini,Geneva,via Capgemini Careers,About Us\n\nAt Capgemini Engineering Switzerla...,[{'items': ['About Us At Capgemini Engineerin...,"[{'link': 'http://www.capgemini.com/', 'text':...","[29 days ago, Full-time]",eyJqb2JfdGl0bGUiOiJEYXRhIEFuYWx5c3QgUlx1MDAyNk...,https://encrypted-tbn0.gstatic.com/images?q=tb...,29 days ago,Full-time,2024-06-04 12:33:40.843725,data analyst,"Geneva, Switzerland"
4,Data analyst budget et reporting (H/F),ROLEX SA,Geneva,via Www.carrieres-Rolex.com,"Introduction\n\nPour la division Achats, souti...",[{'items': ['Introduction Pour la division Ac...,"[{'link': 'http://www.rolex.com/', 'text': 'ro...","[19 days ago, Full-time]",eyJqb2JfdGl0bGUiOiJEYXRhIGFuYWx5c3QgYnVkZ2V0IG...,https://encrypted-tbn0.gstatic.com/images?q=tb...,19 days ago,Full-time,2024-06-04 12:33:40.843725,data analyst,"Geneva, Switzerland"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,Web Analyst,JobCloud AG,Genf,über Letempsemploi.ch,"As a Web Analyst, you'll play a crucial role i...","[{'items': ['As a Web Analyst, you'll play a c...","[{'link': 'http://www.jobcloud.ch/', 'text': '...","[vor 8 Tagen, Vollzeit]",eyJqb2JfdGl0bGUiOiJXZWIgQW5hbHlzdCIsImh0aWRvY2...,NaN,vor 8 Tagen,Vollzeit,2024-06-04 12:33:52.292962,data analyst,"Geneva, Switzerland"
7,Digital Data Analytics Specialist,United Nations Children's Fund,Genf,über BeBee Schweiz,Job Description\n\nDescription\n\n–>\n\nAs par...,[{'items': ['Job Description Description –> ...,"[{'link': 'https://www.unicef.org/', 'text': '...","[vor 24 Tagen, Vollzeit]",eyJqb2JfdGl0bGUiOiJEaWdpdGFsIERhdGEgQW5hbHl0aW...,NaN,vor 24 Tagen,Vollzeit,2024-06-04 12:33:52.292962,data analyst,"Geneva, Switzerland"
8,Analyst Compensation,"Gavi, The Vaccine Alliance",Genf,über BeBee Schweiz,Job Description\n\nDescription\n\n–>\nReportin...,[{'items': ['Job Description Description –> ...,"[{'link': 'http://www.gavi.org/', 'text': 'gav...","[vor 22 Tagen, Vollzeit]",eyJqb2JfdGl0bGUiOiJBbmFseXN0IENvbXBlbnNhdGlvbi...,https://encrypted-tbn0.gstatic.com/images?q=tb...,vor 22 Tagen,Vollzeit,2024-06-04 12:33:52.292962,data analyst,"Geneva, Switzerland"
9,Sustainability analyst,Michael Page,Genf,über Michael Page Switzerland,About Our Client\n\nInternational company base...,[{'items': ['About Our Client International c...,"[{'link': 'http://www.page.com/', 'text': 'pag...",[Vollzeit],eyJqb2JfdGl0bGUiOiJTdXN0YWluYWJpbGl0eSBhbmFseX...,https://encrypted-tbn0.gstatic.com/images?q=tb...,NaN,Vollzeit,2024-06-04 12:33:52.292962,data analyst,

In [5]:
jobs_all.describe()

,date_time
count,61
mean,2024-06-04 12:33:46.681406
min,2024-06-04 12:33:40.843725
25%,2024-06-04 12:33:42.839059
50%,2024-06-04 12:33:47.883009
75%,2024-06-04 12:33:50.501243
max,2024-06-04 12:33:54.193961


In [6]:
title_columns = jobs_all.columns

if not path.exists("DB_data-analyst.csv") :
    jobs_all.to_csv("DB_data-analyst.csv", index=False, mode="w")

else :
    df_actual = pd.read_csv("DB_data-analyst.csv")

    df_actual = pd.concat([df_actual, jobs_all], ignore_index=True)

    #Drop duplicates
    df_actual.drop_duplicates(subset="description", inplace=True)

    df_actual.to_csv("DB_data-analyst.csv", index=False, mode="w")